<a href="https://colab.research.google.com/github/llorenzo62/Use_Moodle/blob/master/Listados_garda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re,zipfile,glob

def unzipODF(pathtofile):
    fzip=zipfile.ZipFile(pathtofile)
    return repr(fzip.read('content.xml').decode())



In [ ]:
def extrae_tablas(direcc):
    hc=unzipODF(direcc)
    tablas=re.findall('<table:table (table:name.*?)</table:table>',hc)
    dtabla={}
    for tabla in tablas:
        nome=re.findall('table:name="(.*?)"',tabla)[0]
        if nome in ['NVScriptsProperties','DO NOT DELETE - AutoCrat Job Se']:
            continue
        #print(nome)
        rows=re.findall('<table:table-row (.*?)</table:table-row>',tabla)
        if rows:
            dtabla[nome]=[re.findall(r'<text:p>(.*?)</text:p>',item)[:4] for item in rows[2:] if item]
            dtabla[nome]=[item for item in dtabla[nome] if item]
    return dtabla

In [ ]:
def decorationList(lista):
  return '%s.- %s (%s)' % tuple(lista)#'  '.join(lista)

In [ ]:

def options(num,options):
    return '''
    <ITEM TYPE="multichoice" REQUIRED="1">
       <ITEMID>
            <![CDATA[{}]]>
       </ITEMID>
       <ITEMTEXT>
            <![CDATA[selecciona un opcion]]>
       </ITEMTEXT>
       <ITEMLABEL>
            <![CDATA[opciones]]>
       </ITEMLABEL>
       <PRESENTATION>
            <![CDATA[d>>>>>{}]]>
       </PRESENTATION>
       <OPTIONS>
            <![CDATA[h]]>
       </OPTIONS>
       <DEPENDITEM>
            <![CDATA[0]]>
       </DEPENDITEM>
       <DEPENDVALUE>
            <![CDATA[]]>
       </DEPENDVALUE>
    </ITEM>
    '''.format(num,options)[1:]

def listado(num,label,listado,Idop,opcion):
    return '''
    <ITEM TYPE="multichoice" REQUIRED="0">
           <ITEMID>
                <![CDATA[{}]]>
           </ITEMID>
           <ITEMTEXT>
                <![CDATA[{}]]>
           </ITEMTEXT>
           <ITEMLABEL>
                <![CDATA[l1]]>
           </ITEMLABEL>
           <PRESENTATION>
                <![CDATA[c>>>>>{}]]>
           </PRESENTATION>
           <OPTIONS>
                <![CDATA[h]]>
           </OPTIONS>
           <DEPENDITEM>
                <![CDATA[{}]]>
           </DEPENDITEM>
           <DEPENDVALUE>
                <![CDATA[{}]]>
           </DEPENDVALUE>
    </ITEM>
    '''.format(num,label,listado,Idop,opcion)[1:]

def head():
    return '''<?xml version="1.0" encoding="UTF-8" ?>
<FEEDBACK VERSION="200701" COMMENT="XML-Importfile for mod/feedback">
     <ITEMS>'''

def tail():
    return '''
    </ITEMS>
</FEEDBACK>'''

def pagebreak(num):
    return '''
      <ITEM TYPE="pagebreak" REQUIRED="0">
           <ITEMID>
                <![CDATA[{}]]>
           </ITEMID>
           <ITEMTEXT>
                <![CDATA[]]>
           </ITEMTEXT>
           <ITEMLABEL>
                <![CDATA[]]>
           </ITEMLABEL>
           <PRESENTATION>
                <![CDATA[]]>
           </PRESENTATION>
           <OPTIONS>
                <![CDATA[]]>
           </OPTIONS>
           <DEPENDITEM>
                <![CDATA[0]]>
           </DEPENDITEM>
           <DEPENDVALUE>
                <![CDATA[]]>
           </DEPENDVALUE>
      </ITEM>'''.format(num)[1:]

In [ ]:
nivel='2º ESO'

In [ ]:

dictinfo={}
f=glob.glob('*.ods')
ops=[]
for item in f:
    ops.append(item.split('/')[-1].split('.')[0])
dictinfo[nivel]=[f,ops]
dictinfo

{'2º ESO': [['2ºESO A.ods',
   '2ºESO E.ods',
   '2ºESO C.ods',
   '2ºESO D.ods',
   '2ºESO B.ods'],
  ['2ºESO A', '2ºESO E', '2ºESO C', '2ºESO D', '2ºESO B']]}

In [ ]:
datos={}
for key,val in dictinfo.items():
    direcc,descrip=val
    datos[key]={}
    for d,s in zip(direcc,descrip):
        datos[key][s]=extrae_tablas(d)
#datos

In [ ]:

keys=sorted(datos[nivel].keys())

In [ ]:
with open('listados_'+nivel+'.xml','w') as fich:
    fich.write(head()+'\n')
    num=1
    Idop=num
    fich.write(options(Idop,'|'.join(keys))+'\n')
    num+=1
    fich.write(pagebreak(num))
    for key in keys:
        lista=''
        key2=list(datos[nivel][key].keys())[0]
        for item in datos[nivel][key][key2]:
            lista+='|'+decorationList(item[:3])
        lista=lista[1:]
        num+=1
        fich.write(pagebreak(num)+'\n')
        num+=1
        fich.write(listado(num,key2,lista,Idop,key)+'\n')
    fich.write(tail())
    
    

In [ ]:
#Limpieza
!rm *.ods
!rm *.xml